Prepare MORO results for robustness analysis

In this notebook, the following tasks are conducted:

- Merge the results of the different seed runs
- Create a first promising policy set by saving all pareto optimal solutions, for both the mean and 90th percentile based MORO results
- Remove duplicate solutions from this set
- Further reduce the number of policies in the pareto efficient sets with the use of k-means clustering
- Combine the remaining optimal solutions from the mean and 90th percentile based sets into one promising policy set
- Test these policies in a large set of experiments, which are sampled from the models uncertainty space
- Save these experiments and outcomes for robustness analysis and scenario discovery

NB: In this notebook, absolute path references/imports are used due to problems with relative paths in the authors Python interpreter environment 

In [ ]:

# LOAD THE CSV RESULTS FROM ALL SEEDS & BOTH MOROs

from pathlib import Path
import pandas as pd
import numpy as np
import re

# Configuration
SEEDS = range(5)                     # possible seeds for MORO runs (0 to 4))
MORO_DIR_MAP = {                     # folder names on disk
    "mean": "archives_mean_seed_{seed}",
    "p90" : "archives_90_seed_{seed}",
}

# Base directory = folder where THIS notebook / script lives
# BASE_DIR = Path(__file__).resolve().parent # When usning Jupyter files

BASE_DIR = Path().resolve().parent # for Jupyter Notebooks
print(f" Base directory = {BASE_DIR}")

records = []

for kind, dir_pattern in MORO_DIR_MAP.items():
    for seed in SEEDS:
        # Absolute path to the archive directory
        archive_dir = (BASE_DIR / dir_pattern.format(seed=seed)).resolve()
        tmp_dir     = archive_dir / "tmp"

        if not tmp_dir.exists():
            print(f"No tmp/ folder for {kind} seed {seed} → {tmp_dir}")
            continue

        # Grab *.csv files and pick the one with the highest NFE number (eg the last generation)
        csv_files = sorted(tmp_dir.glob("*.csv"), key=lambda p: int(re.findall(r"\d+", p.stem)[0]))
        if not csv_files:
            print(f"No CSV files in {tmp_dir}")
            continue

        final_csv = csv_files[-1]           # highest generation (last in sorted list)
        print(f"Using {final_csv.relative_to(BASE_DIR)}")

        df = pd.read_csv(final_csv)

        # Tag bookkeeping columns
        df["moro_kind"] = kind
        df["seed"]      = seed
        records.append(df)


# COMBINE EVERYTHING
if records:
    df_moro = pd.concat(records, ignore_index=True)
    print("Combined shape:", df_moro.shape)
else:
    raise FileNotFoundError("No MORO CSV files were found ─ check paths above.")

df_moro.head()


 Base directory = C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main
No tmp/ folder for mean seed 0 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_0\tmp
No tmp/ folder for mean seed 1 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_1\tmp
Using archives_mean_seed_2\tmp\results_seed_2.csv
Using archives_mean_seed_3\tmp\results_seed_3.csv
No tmp/ folder for mean seed 4 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_4\tmp
No tmp/ folder for p90 seed 0 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_90_seed_0\tmp
Using archives_90_seed_1\tmp\42004.csv
No tmp/ folder for p90 seed 2 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_90_seed_2\tmp
No tmp/ folder for p90 seed 3 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Makin

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Gelderland Expected Number of Deaths,Overijssel Expected Annual Damage,Overijssel Dike Investment Costs,Overijssel Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,moro_kind,seed
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0.001850,1.709083e+05,7.442279e+07,0.000014,1.129600e+09,519.302374,mean,2
1,0,0,0,0,0,0,0,0,0,1,...,4,3,0.039648,1.932565e+05,1.121632e+08,0.000013,2.424000e+08,13526.675536,mean,2
2,0,0,0,0,0,0,0,0,0,1,...,7,0,0.038923,2.121356e+05,9.611556e+07,0.000015,2.424000e+08,11483.530112,mean,2
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0.456134,1.557884e+06,9.014178e+07,0.000960,3.070000e+07,0.000000,mean,2
4,0,0,0,0,0,0,0,0,0,1,...,4,0,0.652514,1.209853e+07,4.596357e+07,0.007557,1.212000e+08,0.000000,mean,2


In [ ]:
# MERGE THE LAST‑GENERATION CSVs FOR MEAN‑BASED MORO RUNS
from pathlib import Path
import pandas as pd
import re

SEEDS = range(5)                                       # 0–4, the possible seeds for MORO runs
BASE_DIR = Path().resolve().parent                     # adjust if needed
records_mean = []

for seed in SEEDS:
    archive_dir = (BASE_DIR / f"archives_mean_seed_{seed}").resolve()
    tmp_dir     = archive_dir / "tmp"

    if not tmp_dir.exists():
        print(f"No tmp/ folder for mean seed {seed} → {tmp_dir}")
        continue

    csv_files = sorted(
        tmp_dir.glob("*.csv"),
        key=lambda p: int(re.findall(r"\d+", p.stem)[0])
    )
    if not csv_files:
        print(f"No CSV files in {tmp_dir}")
        continue

    final_csv = csv_files[-1]  # highest NFE
    print(f"Using {final_csv.relative_to(BASE_DIR)} (mean seed {seed})")

    df = pd.read_csv(final_csv)
    df["seed"] = seed
    records_mean.append(df)

# Combine and save
if records_mean:
    df_mean = pd.concat(records_mean, ignore_index=True)
    print("🟦 Mean‑based merged shape:", df_mean.shape)
    df_mean.to_csv("mean_moro_merged.csv", index=False)
else:
    raise FileNotFoundError("No mean‑based MORO CSV files were found.")


No tmp/ folder for mean seed 0 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_0\tmp
No tmp/ folder for mean seed 1 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_1\tmp
Using archives_mean_seed_2\tmp\results_seed_2.csv (mean seed 2)
Using archives_mean_seed_3\tmp\results_seed_3.csv (mean seed 3)
No tmp/ folder for mean seed 4 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_mean_seed_4\tmp
🟦 Mean‑based merged shape: (2828, 38)


In [ ]:
# MERGE THE LAST‑GENERATION CSVs FOR 90ᵗʰ‑PERCENTILE MORO RUNS

from pathlib import Path
import pandas as pd
import re

SEEDS = range(5)                                       # 0–4, the possible seeds for MORO runs
BASE_DIR = Path().resolve().parent
records_p90 = []

for seed in SEEDS:
    archive_dir = (BASE_DIR / f"archives_90_seed_{seed}").resolve()
    tmp_dir     = archive_dir / "tmp"

    if not tmp_dir.exists():
        print(f"No tmp/ folder for p90 seed {seed} → {tmp_dir}")
        continue

    csv_files = sorted(
        tmp_dir.glob("*.csv"),
        key=lambda p: int(re.findall(r"\d+", p.stem)[0])
    )
    if not csv_files:
        print(f"No CSV files in {tmp_dir}")
        continue

    final_csv = csv_files[-1]  # highest NFE
    print(f"Using {final_csv.relative_to(BASE_DIR)} (p90 seed {seed})")

    df = pd.read_csv(final_csv)
    df["seed"] = seed
    records_p90.append(df)

# Combine and save
if records_p90:
    df_p90 = pd.concat(records_p90, ignore_index=True)
    print(" 90ᵗʰ‑percentile merged shape:", df_p90.shape)
    df_p90.to_csv("p90_moro_merged.csv", index=False)
else:
    raise FileNotFoundError("No p90‑based MORO CSV files were found.")


No tmp/ folder for p90 seed 0 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_90_seed_0\tmp
Using archives_90_seed_1\tmp\42004.csv (p90 seed 1)
No tmp/ folder for p90 seed 2 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_90_seed_2\tmp
No tmp/ folder for p90 seed 3 → C:\Users\tlwal\OneDrive\Documenten\EPA\Model Based Desision Making\MBDM-main\archives_90_seed_3\tmp
Using archives_90_seed_4\tmp\results_seed_4.csv (p90 seed 4)
🟧 90ᵗʰ‑percentile merged shape: (3458, 38)


In [ ]:
import numpy as np
import pandas as pd

# General method Pareto filter

OBJ_COLS = [
    "Gelderland Expected Number of Deaths",
    "Overijssel Expected Annual Damage",
    "Overijssel Dike Investment Costs",
    "Overijssel Expected Number of Deaths",
    "RfR Total Costs",
    "Expected Evacuation Costs",
]

def pareto_efficient(cost_matrix: np.ndarray) -> np.ndarray:
    """Boolean mask of Pareto‑efficient rows (minimise all objectives)."""
    is_eff = np.ones(cost_matrix.shape[0], dtype=bool)
    for i, c in enumerate(cost_matrix):
        if is_eff[i]:
            is_eff[is_eff] = np.any(cost_matrix[is_eff] < c, axis=1)
            is_eff[i] = True
    return is_eff


In [ ]:
# 1.  APPLY PARETO FILTER & DEDUPLICATION  (mean-based)

costs_mean = df_mean[OBJ_COLS].values
mask_mean  = pareto_efficient(costs_mean)
df_pareto_mean = df_mean.loc[mask_mean].copy()

# Infer lever columns: everything except objectives + meta colums
NONLEVER = set(OBJ_COLS + ["seed"])          
LEVER_COLS = [c for c in df_pareto_mean.columns if c not in NONLEVER]

df_mean_unique = df_pareto_mean.drop_duplicates(subset=LEVER_COLS)
print(f"Unique Pareto (mean): {len(df_mean_unique)}")

df_mean_unique.to_csv("promising_policies_mean.csv", index=False)


🟦 Unique Pareto (mean): 1770


In [ ]:
# 2.  APPLY PARETO FILTER & DEDUPLICATION  (p90-based)

costs_p90 = df_p90[OBJ_COLS].values
mask_p90  = pareto_efficient(costs_p90)
df_pareto_p90 = df_p90.loc[mask_p90].copy()

NONLEVER = set(OBJ_COLS + ["seed"])          
LEVER_COLS = [c for c in df_pareto_p90.columns if c not in NONLEVER]

df_p90_unique = df_pareto_p90.drop_duplicates(subset=LEVER_COLS)
print(f"Unique Pareto (p90): {len(df_p90_unique)}")

df_p90_unique.to_csv("promising_policies_p90.csv", index=False)


Unique Pareto (p90): 2399


Both pareto efficient sets contain way too many policies to conduct an easily interpretable final robustness analysis (1770 and 2399 solutions of the mean and p90 based sets respectively). Therefore, it's needed to take small samples from both sets, while trying to save as much variety in the ultimate policy set as possible. To do so, we cluster the policies using k-means clustering. Hereby, the clusters are created in the outcome space and thus use the effects of policies as the measure of policy diversity. This provides the best guarantee to build understanding of what possible trade-offs are made when desiging policies. In order to end up with an easily interpretable and comprehensive set of 30 policies, we want to save 15 solutions from both the mean and p90 based sets. Therefore, we create 15 clusters.   

In [ ]:
# K‑MEANS REDUCTION (15 clusters each) FOR MEAN & P90, THEN MERGE

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min

OBJ_COLS = [
    "Gelderland Expected Number of Deaths",
    "Overijssel Expected Annual Damage",
    "Overijssel Dike Investment Costs",
    "Overijssel Expected Number of Deaths",
    "RfR Total Costs",
    "Expected Evacuation Costs",
]

def kmeans_reduce(df_in: pd.DataFrame, n_clusters: int = 15, label: str = "") -> pd.DataFrame:
    """
    Cluster on objective space and return the representative policy
    closest to each cluster centre.
    """
    scaler = StandardScaler()
    X = scaler.fit_transform(df_in[OBJ_COLS])

    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init='auto')
    labels = kmeans.fit_predict(X)
    df_in = df_in.copy()
    df_in["cluster"] = labels

    closest_idx, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
    df_red = df_in.iloc[closest_idx].copy()
    df_red["source"] = label  # tag origin (mean or p90)
    return df_red

# Reduce each set to 15 representatives 
df_mean15 = kmeans_reduce(df_mean_unique, n_clusters=15, label="mean")
df_p90_15 = kmeans_reduce(df_p90_unique,  n_clusters=15, label="p90")

# Merge the two reduced sets
df_merged = pd.concat([df_mean15, df_p90_15], ignore_index=True)
print(f"Final merged set size: {len(df_merged)} policies.")

# Save results
df_mean15.to_csv("representative_policies_mean_15.csv", index=False)
df_p90_15.to_csv("representative_policies_p90_15.csv", index=False)
df_merged.to_csv("representative_policies_combined_15.csv", index=False)


Final merged set size: 30 policies.


As a next stape, we conduct experiments with the 30 remaining policies in a larger scenario set (N=1000), in order to be able to conduct a meaningfull robustness analysis.

In [ ]:
#  TEST POLICIES UNDER 1000 NEW SCENARIOS
from ema_workbench import (
    Policy,                         
    MultiprocessingEvaluator,
    save_results,                   
)
from ema_workbench.em_framework import sample_uncertainties 
from problem_formulation import get_model_for_problem_formulation
import numpy as np, random, os

# Set up the model with the predefined problem formulation
model, _ = get_model_for_problem_formulation(2)   # Again use PF 2
lever_names = [l.name for l in model.levers]

# Build Policy objects from the promising policies set
policies = []
for _, row in df_merged.iterrows():
    lever_dict = {lv: row[lv] for lv in lever_names}
    policies.append(Policy(f"pol_{len(policies):03d}", **lever_dict))



A.1
A.2
A.3


In [ ]:
# Sample 1000 scenarios
N_SCEN = 2 #100
scenarios = sample_uncertainties(model, N_SCEN)



In [ ]:
# Run experiments
run_label = "promising_resample_1000"
os.makedirs("results", exist_ok=True)

with MultiprocessingEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(
        scenarios=scenarios,
        policies=policies,
    )


100%|██████████████████████████████████████████| 60/60 [00:02<00:00, 26.64it/s]


In [ ]:
# Save results

save_path = f"results/{run_label}.tar.gz"
save_results((experiments, outcomes), save_path)
print(f"✅ Saved: {save_path}")

✅ Saved: results/promising_resample_1000.tar.gz
